In [1]:
import torch
import numpy as np
import torchaudio
from datasets import load_dataset, Audio
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from transformers.models.whisper.feature_extraction_whisper import WhisperFeatureExtractor
from transformers.models.whisper.tokenization_whisper import WhisperTokenizer

c:\Users\follo\anaconda3\envs\GPU\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "openai/whisper-tiny"

processor = WhisperProcessor.from_pretrained(model_name)
feature_extractor = processor.feature_extractor
tokenizer = processor.tokenizer
model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [3]:
data_files = {
    "train": "Dataset/train.csv",
    "validation": "Dataset/val.csv",
    "test": "Dataset/test.csv"
}

dataset = load_dataset("csv", data_files=data_files)

train_ds = dataset["train"]
val_ds   = dataset["validation"]
test_ds  = dataset["test"]

print("Train size:", len(train_ds))
print("Validation size:", len(val_ds))
print("Test size:", len(test_ds))

Train size: 22488
Validation size: 2811
Test size: 2812


In [4]:
def preprocess(batch):
    # Загружаем wav
    waveform, sr = torchaudio.load(batch["path"])
    waveform = waveform.mean(dim=0).numpy()  # моно

    # Приводим к 16kHz
    if sr != 16000:
        waveform = torchaudio.functional.resample(
            torch.tensor(waveform), sr, 16000
        ).numpy()

    # Прогоняем через WhisperProcessor
    inputs = processor(
        waveform,
        sampling_rate=16000,
        return_tensors="pt"
    )

    labels = processor.tokenizer(
        batch["text"],
        return_tensors="pt"
    ).input_ids

    batch["input_features"] = inputs.input_features[0]
    batch["labels"] = labels[0]
    return batch

In [5]:
# Применяем препроцессинг ко всем сплитам
train_ds = train_ds.map(preprocess)
val_ds   = val_ds.map(preprocess)
test_ds  = test_ds.map(preprocess)

In [6]:
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Коллатор специально для Whisper: делает padding
    и для input_features, и для labels.
    """

    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # входные признаки (мел-спектрограммы)
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(
            input_features,
            return_tensors="pt"
        )

        # текстовые метки
        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors="pt"
        )

        # заменяем паддинг на -100 для PyTorch loss
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels
        return batch

In [7]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=100,  # маленький датасет
    logging_steps=10,
    save_steps=100,
    fp16=torch.cuda.is_available(),
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    # processing_class=processor  # можно использовать для скрытия FutureWarning
)

trainer.train()

C:\Users\follo\AppData\Local\Temp\ipykernel_10596\2510436567.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
10,7.732600
20,6.591200
30,6.035800
40,4.222800
50,3.183000
60,2.504000
70,1.887500
80,1.679500
90,1.187200
100,1.188000


TrainOutput(global_step=100, training_loss=3.6211781311035156, metrics={'train_runtime': 311.3659, 'train_samples_per_second': 2.569, 'train_steps_per_second': 0.321, 'total_flos': 1.9695108096e+16, 'train_loss': 3.6211781311035156, 'epoch': 0.03557452863749555})

In [12]:
metrics = trainer.evaluate(val_ds)
print(metrics)

{'eval_loss': 1.2308405637741089, 'eval_runtime': 601.1059, 'eval_samples_per_second': 4.676, 'eval_steps_per_second': 0.586, 'epoch': 0.03557452863749555}


In [13]:
processor.save_pretrained("./whisper_finetuned")

[]